In [1]:
import os
import pandas as pd
import gzip
import numpy as np
from itertools import product
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from collections import defaultdict, Counter, OrderedDict

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to read FASTA file and yield name and sequence
def fasta_iter(fasta_file):
    with gzip.open(fasta_file, 'rt') as file:
        name = None
        seqs = []
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if name:
                    yield name, ''.join(seqs)
                name = line[1:]  # Skip '>'
                seqs = []
            else:
                seqs.append(line)
        if name:
            yield name, ''.join(seqs)  # Yield the last entry

# Function to read FASTQ file and yield name and sequence
def fastq_iter(fastq_file):
    with gzip.open(fastq_file, 'rt') as file:
        while True:
            name = file.readline().strip()[1:]  # Skip '@' in FASTQ
            seq = file.readline().strip()
            file.readline()  # Skip the '+' line
            file.readline()  # Skip the quality line
            if not name or not seq:
                break
            yield name, seq

# Function to generate feature mapping for k-mers with sequences as keys
def generate_feature_mapping(kmer_len):
    BASE_COMPLEMENT = {"A": "T", "T": "A", "G": "C", "C": "G"}
    kmer_hash = {}
    kmer_names = []
    counter = 0
    for kmer in product("ATGC", repeat=kmer_len):
        kmer = ''.join(kmer)
        rev_compl = ''.join([BASE_COMPLEMENT[x] for x in reversed(kmer)])  # Generate reverse complement
        if kmer not in kmer_hash and rev_compl not in kmer_hash:
            # Use canonical k-mer (lexicographically smaller of kmer and rev_compl)
            canonical_kmer = min(kmer, rev_compl)
            kmer_hash[canonical_kmer] = counter
            kmer_names.append(canonical_kmer)  # Store the k-mer for naming features
            counter += 1
    return kmer_hash, kmer_names, counter
    
# Function to detect file type based on file extension
def detect_file_type(file_path):
    ext = os.path.splitext(file_path)[-1].lower()  # Get file extension and convert to lowercase
    # Handle gzipped files
    if ext == '.gz':
        ext = os.path.splitext(file_path[:-3])[-1].lower()  # Check the extension before ".gz"
        
    if ext in ['.fasta', '.fa', '.fna']:
        return 'fasta'
    elif ext in ['.fastq', '.fq']:
        return 'fastq'
    else:
        raise ValueError("Unsupported file extension. Please provide a valid FASTA or FASTQ file.")
        
# Function to generate k-mer features from FASTA or FASTQ
def generate_kmer_features(folder_path, length_threshold, kmer_len, split=False, split_threshold=0):
    # List all files in the folder with .fna.gz extension
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.fna.gz')]
    
    kmer_dict, kmer_names, nr_features = generate_feature_mapping(kmer_len)
    composition = OrderedDict()

    for file_path in files:
        # Automatically detect file type from file extension
        file_type = detect_file_type(file_path)

        # Select the appropriate file iterator based on the file type
        if file_type == 'fasta':
            seq_iter = fasta_iter(file_path)
        elif file_type == 'fastq':
            seq_iter = fastq_iter(file_path)
        else:
            raise ValueError("Unsupported file type. Use 'fasta' or 'fastq'.")

        def seq_list():
            for h, seq in seq_iter:
                # Ambil hanya ID sebelum karakter "|"
                h = h.split('|')[0].strip()
                
                if not split:
                    yield h, seq
                elif len(seq) >= split_threshold:
                    half = len(seq) // 2
                    yield (h + '_1', seq[:half])
                    yield (h + '_2', seq[half:])

        for h, seq in seq_list():
            if len(seq) < length_threshold:
                continue
            norm_seq = str(seq).upper()
            kmers = [kmer_dict[norm_seq[i:i+kmer_len]]
                     for i in range(len(norm_seq) - kmer_len + 1)
                     if norm_seq[i:i+kmer_len] in kmer_dict]  # Ignore kmers with non-canonical bases
            composition[f"{file_path}_{h}"] = np.bincount(np.array(kmers, dtype=np.int64), minlength=nr_features)

    # Create DataFrame with k-mer names as column headers
    df = pd.DataFrame.from_dict(composition, orient='index', dtype=float, columns=kmer_names)

    df = df.apply(lambda x: x + 1e-5)  # Smoothing
    df = df.div(df.sum(axis=1), axis=0)  # Normalize by row (sequence)

    return df

# Contigs

## Feature

In [3]:
fasta_folder = './'  # Updated file path to your uploaded file
kmer_len = 5  # Original k-mer length
length_threshold = 100  # Example sequence length threshold

# Generate spaced k-mer features
kmer_df = generate_kmer_features(fasta_folder, length_threshold, kmer_len)
kmer_df = pd.DataFrame(kmer_df)
kmer_df

,AAAAA,AAAAT,AAAAG,AAAAC,AAATA,AAATT,AAATG,AAATC,AAAGA,AAAGT,...,CCGAG,CGGAG,CCTCG,CGACG,CCACG,CCCCG,CGCCG,CCGCG,CCAGG,CCCGG
./GCF_000009905-Exact.d2fc5d35.fna.gz_r2.1,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,6.992757e-08,...,6.992757e-08,6.992757e-08,6.992757e-08,6.992827e-03,6.992757e-08,6.992827e-03,1.398558e-02,6.992827e-03,6.992827e-03,2.097834e-02
./GCF_000009905-Exact.d2fc5d35.fna.gz_r5.1,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,...,1.315752e-02,6.578792e-03,6.578726e-08,2.631497e-02,6.578726e-08,6.578726e-08,6.578792e-03,6.578726e-08,6.578792e-03,6.578792e-03
./GCF_000009905-Exact.d2fc5d35.fna.gz_r6.1,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,6.060418e-08,...,6.060418e-08,6.060479e-03,6.060418e-08,6.060418e-08,6.060479e-03,1.818131e-02,6.060479e-03,6.060418e-08,2.424173e-02,1.212090e-02
./GCF_000009905-Exact.d2fc5d35.fna.gz_r7.1,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,6.578726e-08,...,6.578726e-08,6.578792e-03,6.578726e-08,6.578726e-08,6.578726e-08,1.973624e-02,6.578726e-08,6.578792e-03,6.578726e-08,1.315752e-02
./GCF_000009905-Exact.d2fc5d35.fna.gz_r8.1,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,6.944198e-08,...,6.944198e-08,6.944198e-08,6.944267e-03,1.388846e-02,6.944198e-08,6.944267e-03,6.944198e-08,6.944267e-03,2.777686e-02,1.388846e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
./GCF_000009045-Exact.d2fc5950.fna.gz_r9996.1,9.258820e-08,9.258913e-03,9.258913e-03,9.258820e-08,9.258820e-08,9.258913e-03,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,...,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08,9.258820e-08
./GCF_000009045-Exact.d2fc5950.fna.gz_r9997.1,7.142596e-08,7.142667e-03,1.428526e-02,7.142596e-08,1.428526e-02,2.857046e-02,7.142596e-08,7.142596e-08,2.142786e-02,7.142596e-08,...,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08,7.142596e-08
./GCF_000009045-Exact.d2fc5950.fna.gz_r9998.1,1.554368e-02,5.181262e-03,5.181262e-03,5.181262e-03,5.181210e-08,5.181262e-03,5.181262e-03,5.181210e-08,2.072489e-02,5.181262e-03,...,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08,5.181210e-08
./GCF_000009045-Exact.d2fc5950.fna.gz_r9999.1,6.328909e-08,6.328909e-08,6.328972e-03,6.328909e-08,6.328909e-08,6.328909e-08,6.328909e-08,6.328909e-08,6.328909e-08,6.328972e-03,...,6.328909e-08,6.328972e-03,6.328909e-08,6.328909e-08,6.328909e-08,6.328909e-08,6.328972e-03,1.265788e-02,6.328909e-08,6.328909e-08


## Label

In [4]:
# label_path = "./contigs/gsa_mapping.tsv.gz"
# label_df = pd.read_csv(label_path, sep='\t')
# label_df = label_df[["genome_id"]]
# label_df

import re
import pandas as pd
from collections import OrderedDict

def generate_label_dataframe(file_path):
    labels = OrderedDict()
    
    # Automatically detect file type
    file_type = detect_file_type(file_path)
    if file_type == 'fasta':
        seq_iter = fasta_iter(file_path)
    elif file_type == 'fastq':
        seq_iter = fastq_iter(file_path)
    else:
        raise ValueError("Unsupported file type. Use 'fasta' or 'fastq'.")

    for h, seq in seq_iter:
        # Ambil hanya ID sebelum karakter "|"
        seq_id = h.split('|')[0].strip()
        # Ekstrak label di antara tanda petik dua ("")
        label_match = re.search(r'"(.*?)"', h)
        label = label_match.group(1) if label_match else "Unknown"
        
        # Simpan ID dan label
        labels[seq_id] = label

    # Buat DataFrame dari dictionary labels
    label_df = pd.DataFrame.from_dict(labels, orient='index', columns=['Label'])
    
    return label_df

folder_path = './'
files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.fna.gz')]

# Menggabungkan semua label dari beberapa file
label_df = pd.concat([generate_label_dataframe(f) for f in files])

# Reset indeks jika diperlukan
label_df.reset_index(inplace=True)
label_df.columns = ['Sequence_ID', 'Label']
label_df =  label_df[['Label']]
label_df

,Label
0,NC_006177.1 Symbiobacterium thermophilum IAM 1...
1,NC_006177.1 Symbiobacterium thermophilum IAM 1...
2,NC_006177.1 Symbiobacterium thermophilum IAM 1...
3,NC_006177.1 Symbiobacterium thermophilum IAM 1...
4,NC_006177.1 Symbiobacterium thermophilum IAM 1...
...,...
49995,NC_000964.3 Bacillus subtilis subsp. subtilis ...
49996,NC_000964.3 Bacillus subtilis subsp. subtilis ...
49997,NC_000964.3 Bacillus subtilis subsp. subtilis ...
49998,NC_000964.3 Bacillus subtilis subsp. subtilis ...


In [5]:
# Tampilkan statistik deskriptif dari data fitur
print("Statistik Deskriptif Data Fitur:")
print(kmer_df.describe())

# Tampilkan distribusi kelas dari data label
print("\nDistribusi Kelas pada Data Label:")
print(label_df.value_counts())

Statistik Deskriptif Data Fitur:
              AAAAA         AAAAT         AAAAG         AAAAC         AAATA  \
count  5.000000e+04  5.000000e+04  5.000000e+04  5.000000e+04  5.000000e+04   
mean   6.408442e-03  5.242450e-03  3.859990e-03  3.408019e-03  4.399623e-03   
std    1.006083e-02  7.074681e-03  5.794301e-03  5.101668e-03  6.553133e-03   
min    4.015982e-08  4.015982e-08  4.015982e-08  4.291751e-08  4.015982e-08   
25%    6.756523e-08  6.802484e-08  6.711179e-08  6.666439e-08  6.666439e-08   
50%    8.064183e-08  8.546635e-08  7.812188e-08  7.751630e-08  7.873698e-08   
75%    1.001236e-02  7.751708e-03  6.802552e-03  6.578792e-03  7.092012e-03   
max    1.006677e-01  5.426149e-02  4.265304e-02  4.026714e-02  5.426149e-02   

              AAATT         AAATG         AAATC         AAAGA         AAAGT  \
count  5.000000e+04  5.000000e+04  5.000000e+04  5.000000e+04  5.000000e+04   
mean   4.072027e-03  3.433256e-03  3.112894e-03  3.312553e-03  2.658966e-03   
std    6.285606e-0

## Classifier

In [6]:
from sklearn.model_selection import train_test_split
import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


### Pembagian Data

In [7]:
features = kmer_df
labels = label_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome',
       'NC_003910.7 Colwellia psychrerythraea 34H, complete sequence',
       'NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence',
       'NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence',
       'NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome'],
      dtype=object), array([8050, 8007, 7957, 8002, 7984]))

Jumlah data uji per kelas:
(array(['NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome',
       'NC_003910.7 Colwellia psychrerythraea 34H, complete sequence',
       'NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence',
       'NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence',
       'NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome'],
      dtype=object), array([1950, 1993, 2043, 1998, 2016]))


### Model

In [8]:
# List model yang akan digunakan
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "Support Vector Machine": SVC(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    #"Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    #"MLP Classifier": MLPClassifier(random_state=42, max_iter=500)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

# Tampilkan hasil dalam bentuk dataframe untuk mempermudah pembacaan
import pandas as pd
results_df = pd.DataFrame(results)
results_df

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Classification Report untuk Random Forest:
                                                                                       precision    recall  f1-score   support

               NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.82      0.76      0.79      1950
                         NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.76      0.79      0.78      1993
                NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.97      0.94      0.96      2043
                          NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.83      0.83      0.83      1998
NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.79      0.84      0.82      2016

                                                                             accuracy                           0.83     10000
                                                                 

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Report untuk Logistic Regression:
                                                                                       precision    recall  f1-score   support

               NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.83      0.81      0.82      1950
                         NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.78      0.77      0.77      1993
                NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.97      0.95      0.96      2043
                          NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.86      0.83      0.85      1998
NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.76      0.82      0.79      2016

                                                                             accuracy                           0.84     10000
                                                           

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Classification Report untuk Support Vector Machine:
                                                                                       precision    recall  f1-score   support

               NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.89      0.89      0.89      1950
                         NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.87      0.87      0.87      1993
                NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.98      0.98      0.98      2043
                          NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.93      0.90      0.91      1998
NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.88      0.90      0.89      2016

                                                                             accuracy                           0.91     10000
                                                        

/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



Classification Report untuk K-Nearest Neighbors:
                                                                                       precision    recall  f1-score   support

               NC_000964.3 Bacillus subtilis subsp. subtilis str. 168 complete genome       0.73      0.79      0.76      1950
                         NC_003910.7 Colwellia psychrerythraea 34H, complete sequence       0.75      0.64      0.69      1993
                NC_006177.1 Symbiobacterium thermophilum IAM 14863, complete sequence       0.96      0.96      0.96      2043
                          NC_007645.1 Hahella chejuensis KCTC 2396, complete sequence       0.87      0.78      0.82      1998
NC_007795.1 Staphylococcus aureus subsp. aureus NCTC 8325 chromosome, complete genome       0.72      0.84      0.78      2016

                                                                             accuracy                           0.80     10000
                                                           

,Model,Accuracy,Precision,Recall,F1 Score,Time (seconds)
0,Random Forest,0.8342,0.835848,0.8342,0.834616,58.950899
1,Logistic Regression,0.8372,0.839085,0.8372,0.837801,8.912699
2,Support Vector Machine,0.9094,0.909803,0.9094,0.909527,227.546965
3,K-Nearest Neighbors,0.8028,0.806398,0.8028,0.802153,1.591997


In [ ]:
# List model yang akan digunakan
models = {
    # "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    # "Logistic Regression": LogisticRegression(random_state=42),
    # "Support Vector Machine": SVC(random_state=42),
    # "K-Nearest Neighbors": KNeighborsClassifier(),
    # "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "MLP Classifier": MLPClassifier(random_state=42, max_iter=100)
}

# Menyimpan hasil metrik dan waktu komputasi
results = []

# Loop melalui semua model dan evaluasi performanya
for model_name, model in models.items():
    start_time = time.time()  # Mulai penghitungan waktu
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi menggunakan data testing
    y_pred = model.predict(X_test)
    
    # Hitung waktu komputasi
    elapsed_time = time.time() - start_time
    
    # Evaluasi model dengan berbagai metrik
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Simpan hasil dalam dictionary
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "Time (seconds)": elapsed_time
    })
    
    # Tampilkan hasil classification report untuk setiap model
    print(f"\nClassification Report untuk {model_name}:")
    print(classification_report(y_test, y_pred))

dff = pd.DataFrame(results)
dff

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
hasil = pd.concat([results_df, dff], ignore_index=True)
hasil

# Reads

## Feature

In [9]:
read_file = './reads/anonymous_reads.fq.gz'  # Updated file path to your uploaded file
kmer_len = 4  # Original k-mer length
length_threshold = 100  # Example sequence length threshold

# Generate spaced k-mer features
featureR_df = generate_kmer_features(read_file, length_threshold, kmer_len)
print(featureR_df)

                     AAAA          AAAT          AAAG          AAAC  \
S0R0/1       1.162772e-07  1.162772e-07  1.162772e-07  1.162784e-02   
S0R0/2       1.449247e-07  1.449247e-07  1.449247e-07  1.449247e-07   
S0R1/1       1.351327e-07  1.351327e-07  1.351327e-07  1.351340e-02   
S0R1/2       1.190457e-07  1.190457e-07  1.190457e-07  1.190457e-07   
S0R2/1       1.851805e-07  1.851805e-07  1.851805e-07  1.851805e-07   
...                   ...           ...           ...           ...   
S0R333324/2  1.666629e-07  1.666629e-07  1.666629e-07  1.666629e-07   
S0R333325/1  1.136346e-07  1.136346e-07  1.136346e-07  1.136346e-07   
S0R333325/2  1.333309e-07  1.333309e-07  1.333322e-02  1.333322e-02   
S0R333326/1  1.408424e-07  1.408424e-07  1.408438e-02  1.408424e-07   
S0R333326/2  3.260832e-02  2.173892e-02  1.086951e-02  1.086951e-02   

                     AATA          AATT          AATG          AATC  \
S0R0/1       1.162772e-07  1.162772e-07  1.162772e-07  1.162772e-07   
S0R0/

In [13]:
labelR_path = "./reads/reads_mapping.tsv.gz"
labelR_df = pd.read_csv(labelR_path, sep='\t')
labelR_df = labelR_df[["genome_id"]]
labelR_df

,genome_id
0,Genome19.0
1,Genome19.0
2,Genome19.0
3,Genome19.0
4,Genome18.0
...,...
666649,Genome18.0
666650,Genome19.0
666651,Genome19.0
666652,Genome19.0


In [16]:
features = featureR_df
labels = labelR_df

# Pisahkan data menjadi training dan testing set (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Jumlah data latih per kelas:")
print(np.unique(y_train, return_counts=True))

print("\nJumlah data uji per kelas:")
print(np.unique(y_test, return_counts=True))

Jumlah data latih per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0', 'Genome8.0', 'Genome9.0'], dtype=object), array([  2843,   9214,     37,    438,   2081,   8187,   5417,    338,
         1936,  69103, 367831,  12720,    853,  13194,   2856,    444,
         9619,   5141,   1501,    340,   7383,    656,  10800,    391]))

Jumlah data uji per kelas:
(array(['Genome10.0', 'Genome10.1', 'Genome11.0', 'Genome12.0',
       'Genome13.0', 'Genome14.0', 'Genome15.0', 'Genome16.0',
       'Genome17.0', 'Genome18.0', 'Genome19.0', 'Genome2.0',
       'Genome20.0', 'Genome22.0', 'Genome23.0', 'Genome24.0',
       'Genome3.0', 'Genome4.0', 'Genome4.1', 'Genome5.0', 'Genome6.0',
       'Genome7.0'

In [17]:
# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model
rf_model.fit(X_train, y_train)

# Prediksi menggunakan data testing
y_pred = rf_model.predict(X_test)

# Tampilkan hasil akurasi dan classification report
print("\nAkurasi Model:")
print(accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Akurasi Model:
0.6923746165557897

Classification Report:


/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

  Genome10.0       0.00      0.00      0.00       759
  Genome10.1       0.00      0.00      0.00      2264
  Genome11.0       0.00      0.00      0.00         9
  Genome12.0       0.00      0.00      0.00       106
  Genome13.0       0.00      0.00      0.00       491
  Genome14.0       0.00      0.00      0.00      2001
  Genome15.0       0.00      0.00      0.00      1323
  Genome16.0       0.00      0.00      0.00        94
  Genome17.0       0.00      0.00      0.00       446
  Genome18.0       0.14      0.00      0.00     17141
  Genome19.0       0.69      1.00      0.82     92345
   Genome2.0       0.00      0.00      0.00      3170
  Genome20.0       0.00      0.00      0.00       189
  Genome22.0       0.00      0.00      0.00      3268
  Genome23.0       0.00      0.00      0.00       730
  Genome24.0       0.00      0.00      0.00       118
   Genome3.0       0.00      0.00      0.00      2399
   Genome4.0       0.00    

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
